In [1]:
import numpy as np
import pandas as pd
import xarray as xr

from datetime import date
import regionmask

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.io.shapereader as shpreader


In [9]:
!pwd

/home/565/dy9345


In [2]:
from dask.distributed import Client
 
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 4
Total threads: 8,Total memory: 22.46 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44823,Workers: 4
Dashboard: /proxy/8787/status,Total threads: 8
Started: Just now,Total memory: 22.46 GiB
Comm: tcp://127.0.0.1:45169,Total threads: 2
Dashboard: /proxy/36247/status,Memory: 5.62 GiB
Nanny: tcp://127.0.0.1:46105,


### 1. get data: apcp, mjo, nino34

In [2]:
%%time
filelist=[]
for i in range(1981,2014,1):
    filelist.append(f'/g/data/ua8/LE_models/20CRv3/mean_daily/apcp/apcp.{i}.nc')
filelist
ds1=xr.open_mfdataset(filelist,combine='by_coords')

CPU times: user 1.34 s, sys: 627 ms, total: 1.97 s
Wall time: 4.2 s


In [3]:
ds=ds1['apcp'].loc[:,-45:0,100:160]
ds

<xarray.DataArray 'apcp' (time: 12053, lat: 46, lon: 61)>
dask.array<getitem, shape=(12053, 46, 61), dtype=float32, chunksize=(366, 46, 61), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 -45.0 -44.0 -43.0 -42.0 -41.0 ... -3.0 -2.0 -1.0 0.0
  * lon      (lon) float32 100.0 101.0 102.0 103.0 ... 157.0 158.0 159.0 160.0
  * time     (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2013-12-31
Attributes:
    long_name:         daily mean 3-hourly accumulated total precipitation at...
    units:             kg/m^2
    GRIB_name:         APCP
    var_desc:          Precipitation amount
    dataset:           NOAA/CIRES/DOE 20th Century Reanalysis version 3mo Dai...
    level_desc:        Surface
    statistic:         Mean
    parent_stat:       Individual Obs
    standard_name:     precipitation_amount
    valid_range:       [  0. 100.]
    statistic_method:  Ensemble mean is calculated by averaging over all 80 e...
    GridType:          Cylindrical Equidistant Projection Grid
    datum:             wgs84
    actual_range:      [ 0.    35.625]

In [4]:
df=pd.read_csv("/g/data/w40/dy9345/MJO/MJO_ot12.csv")
df['date']=pd.to_datetime(df[['year','month','day']])
df=df.set_index('date').drop(columns=['year','month','day'])
df.loc[df['amplitude'].lt(1) ,'phase'] = 0

In [5]:
dateparser = lambda d: pd.datetime.strptime(d,'%Y%m%d')
df2=pd.read_csv('http://climexp.knmi.nl/data/inino34_daily.dat',
                skiprows=12,delim_whitespace=True,
                names=['date','enso'],
                parse_dates=['date'],
                date_parser=dateparser)
df2=df2.set_index(df2['date']).drop(columns='date')

/local/v45/dy9345/tmp/ipykernel_1741405/1824527752.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  dateparser = lambda d: pd.datetime.strptime(d,'%Y%m%d')


### 1. test 1 grid

In [8]:
def test_1grid(dfmjo,dfenso,dsrain,THR,lat,lon):
    rain=ds.loc[:,lat,lon].to_dataframe().drop(columns=['lat','lon'])
    data=rain.merge(dfmjo[['phase','amplitude']],left_index=True,right_index=True,how='inner')\
    .merge(dfenso['enso'],left_index=True,right_index=True,how='inner')
    perc=np.percentile(rain[rain.apcp != 0]['apcp'],THR)
    r=[]
    for i in range(1,9,1):
        nm1=data.loc[ (data['phase'] == i) & (data['apcp'] >= perc)]
        dm1=data.loc[(data['phase'] == i)]
        r1=nm1.count()/dm1.count()
        r.append(r1['apcp'])
    return perc,r

In [7]:
%%time
ratio=test_1grid(df,df2,ds,67,-20,140)
pvalue=ratio[0]
r=ratio[1]
xx=np.array(r)
xx

CPU times: user 18.9 s, sys: 1.63 s, total: 20.5 s
Wall time: 31.8 s


array([0.04746835, 0.07734807, 0.08568207, 0.10453401, 0.12783505,
       0.1511254 , 0.14361001, 0.07960742])

### 2. loop over grids

In [33]:
ds.shape

(12053, 46, 61)

In [36]:
P67=np.empty((ds.shape[1],ds.shape[2]))
R=np.empty((ds.shape[1],ds.shape[2],8))
P67[:]=np.NaN
R[:]=np.NaN

for i,lat in enumerate(ds['lat']):
    for j,lon in enumerate(ds['lon']):
        ratio=test_1grid(df,df2,ds,67,lat,lon)
        P67[i, j] = ratio[0]
        R[i, j, :] = np.array(ratio[1])

KeyboardInterrupt: 